## Fixed Income In Python
By Teddy Tenetcha

### Duration and Convexity

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Read and parse csv file. Remove columns "issuance date" and "first coupon payment date"
data = pd.read_csv('ps2-data.csv')
data['Maturity Date'] = pd.to_datetime(data['Maturity Date'], format='%Y-%m-%d')
data['Calendar Date'] = pd.to_datetime(data['Calendar Date'], format='%Y-%m-%d')
data = data.drop(columns = ['Issuance Date'])
data.head()

,Maturity Date,Coupon Rate,Type,Calendar Date,Price
0,2024-01-02,0.0,4,2023-11-15,99.296000
1,2024-01-04,0.0,4,2023-11-15,99.270833
2,2024-01-09,0.0,4,2023-11-15,99.197153
3,2024-01-11,0.0,4,2023-11-15,99.166375
4,2024-01-16,0.0,4,2023-11-15,99.098417


In [11]:
# Extract notes using the value of Type column.
notes = data.loc[data['Type'] == 2].copy()
# Calculate and add a new column T to notes, representing the time to maturity in years, rounded to 1 decimal place.
notes.loc[:, 'T'] = ((notes['Maturity Date']-notes['Calendar Date']).dt.days/365).round(1)

# Create arrays price, c and T corresponding to the Price, Coupon Rate and T for notes.
# Create array B with same length as price and initialized to all zeros.
price = np.array(notes['Price'])
c =  np.array(notes['Coupon Rate'])
T =  np.array(notes['T'])
B = np.zeros(len(price))

# Bootstrap the ZCB prices and add them as a column into the dataframe
B[0] = price[0]/(c[0]/2+100)
for i in range(1,len(T)):
    B[i] = (price[i]-c[i]/2*B[0:i].sum())/(c[i]/2 + 100)
notes.loc[:,'B'] = B
notes.head(14)

,Maturity Date,Coupon Rate,Type,Calendar Date,Price,T,B
30,2024-05-15,0.250,2,2023-11-15,97.507812,0.5,0.973861
38,2024-11-15,0.750,2,2023-11-15,95.656250,1.0,0.949350
39,2025-05-15,2.750,2,2023-11-15,96.687500,1.5,0.927675
40,2025-11-15,4.500,2,2023-11-15,99.234375,2.0,0.907774
41,2026-05-15,3.625,2,2023-11-15,97.386719,2.5,0.889617
42,2026-11-15,4.625,2,2023-11-15,99.859375,3.0,0.870961
43,2027-05-15,2.375,2,2023-11-15,92.781250,3.5,0.852152
44,2027-11-15,2.250,2,2023-11-15,91.546875,4.0,0.834404
45,2028-05-15,2.875,2,2023-11-15,93.203125,4.5,0.816708
46,2028-11-15,3.125,2,2023-11-15,93.652344,5.0,0.798692


In [20]:
# here we will create a function called duration 
# and will apply it to every row of our dataset

def duration (row):
    time_to_maturity = row['T']
    price_row = row['Price']
    coupon_row = row['Coupon Rate']
    
    T_row = np.arange(0.5, time_to_maturity+0.5, 0.5)
    N_row = len(T_row)
    # take ZCB prices for those dates
    B_row = B[:N_row]
    
    payments_row = np.ones(N_row)*coupon_row/2

    payments_row[-1] = payments_row[-1]+100
    
    D_row = 0
    
    return D_row

# test whether your function is working correctly
# print out a duration of a 5y 3.125% coupon treasuty note
test_row = notes.loc[46]
print(duration(test_row))

# apply the function to the whole dataset
notes['Duration'] = notes.apply(duration, axis=1)
notes.head(20)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
0


,Maturity Date,Coupon Rate,Type,Calendar Date,Price,T,B,Duration,Convexity,K Hedge
30,2024-05-15,0.250,2,2023-11-15,97.507812,0.5,0.973861,0,0,1
38,2024-11-15,0.750,2,2023-11-15,95.656250,1.0,0.949350,0,0,1
39,2025-05-15,2.750,2,2023-11-15,96.687500,1.5,0.927675,0,0,1
40,2025-11-15,4.500,2,2023-11-15,99.234375,2.0,0.907774,0,0,1
41,2026-05-15,3.625,2,2023-11-15,97.386719,2.5,0.889617,0,0,1
42,2026-11-15,4.625,2,2023-11-15,99.859375,3.0,0.870961,0,0,1
43,2027-05-15,2.375,2,2023-11-15,92.781250,3.5,0.852152,0,0,1
44,2027-11-15,2.250,2,2023-11-15,91.546875,4.0,0.834404,0,0,1
45,2028-05-15,2.875,2,2023-11-15,93.203125,4.5,0.816708,0,0,1
46,2028-11-15,3.125,2,2023-11-15,93.652344,5.0,0.798692,0,0,1


In [22]:
# here we will create a function called convexity
# and will apply it to every row of our dataset
# it is very similar to the function duration and differs only in one line
def convexity (row):
    time_to_maturity = row['T']
    price_row = row['Price']
    coupon_row = row['Coupon Rate']
    T_row = np.arange(0.5, time_to_maturity+0.5, 0.5)
    N_row = len(T_row)
    B_row = B[:N_row]
    
    payments_row = np.ones(N_row)*coupon_row/2
    payments_row[-1] = payments_row[-1]+100
    
    C_row = 0
    return C_row
# test and print out a duration of a 5y 3.125% coupon treasuty note
test_row = notes.loc[46]
print(convexity(test_row))

# apply the function to the whole dataset
notes['Convexity'] = notes.apply(convexity, axis=1)
notes.head(14)

0


,Maturity Date,Coupon Rate,Type,Calendar Date,Price,T,B,Duration,Convexity,K Hedge
30,2024-05-15,0.250,2,2023-11-15,97.507812,0.5,0.973861,0,0,1
38,2024-11-15,0.750,2,2023-11-15,95.656250,1.0,0.949350,0,0,1
39,2025-05-15,2.750,2,2023-11-15,96.687500,1.5,0.927675,0,0,1
40,2025-11-15,4.500,2,2023-11-15,99.234375,2.0,0.907774,0,0,1
41,2026-05-15,3.625,2,2023-11-15,97.386719,2.5,0.889617,0,0,1
42,2026-11-15,4.625,2,2023-11-15,99.859375,3.0,0.870961,0,0,1
43,2027-05-15,2.375,2,2023-11-15,92.781250,3.5,0.852152,0,0,1
44,2027-11-15,2.250,2,2023-11-15,91.546875,4.0,0.834404,0,0,1
45,2028-05-15,2.875,2,2023-11-15,93.203125,4.5,0.816708,0,0,1
46,2028-11-15,3.125,2,2023-11-15,93.652344,5.0,0.798692,0,0,1


## One Factor Hedging

Suppose we own $50 mn (face value) of the Notes maturing
on 11/15/2028 (with 3.125 coupon rate, duration 4.65, convexity 22.66, and price
$93.652344) and are worried that the price of these notes might go down when the
interest rates go up.


Decide to hedge your position using the Note maturing on 11/15/2025 (with
4.5 coupon rate, duration 1.93, and price $99.234375). Compute the duration
hedge, i.e., what is the face value of the hedging position.

In [25]:
# this is the note with 50mn face value
# that we are trying to hedge
note = notes.loc[46]
f_note = 5*10**6
p_note = note['Price']
d_note = note['Duration']
c_note = note['Convexity']

# here we will create a function called duration_hedge
# and will apply it to every row of our dataset
def duration_hedge (row):
    # every row is a security that we could use to hedge the original position
    price_hedge = row['Price']
    dur_hedge = row['Duration']
    
    k_hedge = 1
    
    return k_hedge
    
notes['K Hedge'] = notes.apply(duration_hedge, axis=1)
notes.head(14)

,Maturity Date,Coupon Rate,Type,Calendar Date,Price,T,B,Duration,Convexity,K Hedge
30,2024-05-15,0.250,2,2023-11-15,97.507812,0.5,0.973861,0,0,1
38,2024-11-15,0.750,2,2023-11-15,95.656250,1.0,0.949350,0,0,1
39,2025-05-15,2.750,2,2023-11-15,96.687500,1.5,0.927675,0,0,1
40,2025-11-15,4.500,2,2023-11-15,99.234375,2.0,0.907774,0,0,1
41,2026-05-15,3.625,2,2023-11-15,97.386719,2.5,0.889617,0,0,1
42,2026-11-15,4.625,2,2023-11-15,99.859375,3.0,0.870961,0,0,1
43,2027-05-15,2.375,2,2023-11-15,92.781250,3.5,0.852152,0,0,1
44,2027-11-15,2.250,2,2023-11-15,91.546875,4.0,0.834404,0,0,1
45,2028-05-15,2.875,2,2023-11-15,93.203125,4.5,0.816708,0,0,1
46,2028-11-15,3.125,2,2023-11-15,93.652344,5.0,0.798692,0,0,1


## Hedging position 

Want to hedge duration and convexity at the same time and use
 .5-year Notes (maturing on 2023-11-15 with 2.375 coupon rate, duration 3.37, convexity 11.64, and price$92.781250)  

In [43]:


import pandas as pd
import numpy as np

# Provided data (replace with actual data if available from the CSV)
note_to_hedge = {
    'duration': 4.65,
    'convexity': 22.66,
    'face_value': 50000000,  # $50 million
    'price': 93.652344
}

hedging_instruments = {
    '3.5-year': {
        'duration': 3.37,
        'convexity': 11.64,
        'price': 92.781250
    },
    '7-year': {
        'duration': 6.77,
        'convexity': 46.87,
        'price': 78.390625
    }
}

# Set up the equations for duration and convexity hedging
# D_portfolio = w1 * D1 + w2 * D2 = 0  (duration neutral)
# C_portfolio = w1 * C1 + w2 * C2 = 0  (convexity neutral)

# where:
# w1 = Face value of 3.5-year note / Price of 3.5-year note
# w2 = Face value of 7-year note / Price of 7-year note

# We want to create a portfolio where the duration and convexity of the portfolio is equal to the note to hedge.
# The portfolio will be the sum of the original note and the hedging instruments.
# w_portfolio = w1 + w2
# D_portfolio = w_portfolio * D_note
# C_portfolio = w_portfolio * C_note


A = np.array([
    [hedging_instruments['3.5-year']['duration'], hedging_instruments['7-year']['duration']],
    [hedging_instruments['3.5-year']['convexity'], hedging_instruments['7-year']['convexity']]
])

# Target duration and convexity (we want the portfolio to match the note)
b = np.array([note_to_hedge['duration'], note_to_hedge['convexity']])


# Solve for w1 and w2 (face values of hedging instruments)
try:
  w = np.linalg.solve(A, b)
  fv_35yr = w[0] * hedging_instruments['3.5-year']['price']
  fv_7yr = w[1] * hedging_instruments['7-year']['price']

  print("Hedging Position:")
  print(f"- 3.5-year Notes Face Value: {fv_35yr:.2f}")
  print(f"- 7-year Notes Face Value: {fv_7yr:.2f}")

except np.linalg.LinAlgError:
  print("The matrix is singular, there may be no unique solution.")

Hedging Position:
- 3.5-year Notes Face Value: 75.65
- 7-year Notes Face Value: 22.03


## Vasicek Model

parameters γ = 1, ¯r = 5%, σ = 20%, γ∗ = 1.5, ¯r∗ = 10% and
suppose that the current repo rate r(0) = 5.5%

In [55]:
# predicted price of a 1.5y Treasury note with a coupon rate of 5%

import numpy as np 

def price(params, current_r, T):
    rn_gamma = params['rn_gamma']
    rn_r_bar = params['rn_r_bar']
    sigma = params['sigma']
    
    beta = (1-np.exp(-rn_gamma*T))/rn_gamma
    alpha = (rn_r_bar-(sigma**2/(2*rn_gamma**2)))*(beta-T) - (sigma**2/(4*rn_gamma)*beta**2)
    print(f'B =',np.exp(alpha-current_r*beta),'beta =',beta,'alpha =', alpha)
    return [alpha,beta]


def c_bond_price(discount_fatcor,coupon_rate,face_value):
    b_price = 0
    for i in range(1,len(discount_fatcor)+1):
        b_price += (coupon_rate * face_value)/2 * discount_fatcor[i-1]
        print(f'Discount factor =',discount_fatcor[i-1], 'PV of coupon =',(coupon_rate * face_value)/2 * discount_fatcor[i-1])
    b_price += face_value * discount_fatcor[-1]
    print(f'PV of principle =',face_value * discount_fatcor[-1])

    return print(f'Coupon Bond Price',b_price)

In [57]:
params = {
    'gamma' : 1,
    'r_bar': 0.05,
    'rn_gamma' : 1.5,
    'rn_r_bar': 0.1,
    'sigma': 0.2
}

for i in range(1,4):
    print(price(params,0.055,i/2))

#for i in range(1,4):
#    print(i/2)

B = 0.9668826466873988 beta = 0.3517556315059902 alpha = -0.014331589291428746
[-0.014331589291428746, 0.3517556315059902]
B = 0.9284888411192138 beta = 0.5179132265677134 alpha = -0.045711688981077976
[-0.045711688981077976, 0.5179132265677134]
B = 0.8891393809001178 beta = 0.5964005169587571 alpha = -0.08469924341017106
[-0.08469924341017106, 0.5964005169587571]


Want to construct a model predicted replicating portfolio for a risk-free
5y zero-coupon bond that consists of cash and the 1.5y Treasury note with a coupon
rate of 5%. How many of the 1.5y notes you need to have in your replicating portfolio?

In [62]:
price_onehalf = 95.8752152617786
B = [0.9668826466873988,0.9284888411192138,0.8891393809001178]
beta_onehalf = [0.3517556315059902,0.5179132265677134,0.5964005169587571]
alpha_onehalf = [-0.014331589291428746,-0.045711688981077976,-0.08469924341017106]

def dp_over_dt(discount_factor, coupon_rate, beta):
    discount_factor = np.array(discount_factor)  
    beta = np.array(beta)  
    
    sensitivity_to_spot_rate = -((coupon_rate * 100) / 2) * np.dot(beta, discount_factor)  
    sensitivity_to_spot_rate -=  100 * beta[-1] * discount_factor[-1]
    
    return sensitivity_to_spot_rate


In [64]:
dp_over_dt(B,0.05,beta_onehalf)/100

-0.564064842765497

In [66]:
params = {
    'gamma' : 1,
    'r_bar': 0.05,
    'rn_gamma' : 1.5,
    'rn_r_bar': 0.1,
    'sigma': 0.2
}

for i in range(1,11):
    print(price(params,0.055,i/2))

B = 0.9668826466873988 beta = 0.3517556315059902 alpha = -0.014331589291428746
[-0.014331589291428746, 0.3517556315059902]
B = 0.9284888411192138 beta = 0.5179132265677134 alpha = -0.045711688981077976
[-0.045711688981077976, 0.5179132265677134]
B = 0.8891393809001178 beta = 0.5964005169587571 alpha = -0.08469924341017106
[-0.08469924341017106, 0.5964005169587571]
B = 0.8504224961836909 beta = 0.6334752877547574 alpha = -0.12718085782809527
[-0.12718085782809527, 0.6334752877547574]
B = 0.812942328202252 beta = 0.6509881694293272 alpha = -0.17129075965246315
[-0.17129075965246315, 0.6509881694293272]
B = 0.7769151041937183 beta = 0.6592606689745052 alpha = -0.21616485880225647
[-0.21616485880225647, 0.6592606689745052]
B = 0.742395549623465 beta = 0.6631683210672125 alpha = -0.26139883444987916
[-0.26139883444987916, 0.6631683210672125]
B = 0.709369780446634 beta = 0.6650141652155558 alpha = -0.3068025572132761
[-0.3068025572132761, 0.6650141652155558]
B = 0.6777951724282917 beta = 0.6

In [68]:
B_fiveyr = [0.9668826466873988,0.9284888411192138,0.8891393809001178, 0.8504224961836909,0.812942328202252,0.7769151041937183,0.742395549623465,0.709369780446634,0.6777951724282917,0.6476178574086983]
beta_fiveyr = [0.3517556315059902,0.5179132265677134,0.5964005169587571,0.6334752877547574,0.6509881694293272,0.6592606689745052,0.6631683210672125,0.6650141652155558,0.6658860802528058,0.6662979437532348]
dp_over_dt(B_fiveyr,0,beta_fiveyr)/100

-0.43150644672929134

In [70]:
delta = (dp_over_dt(B_fiveyr,0,beta_fiveyr)/100) / (dp_over_dt(B,0.05,beta_onehalf)/100)
delta

0.7649944013771567